In [70]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_TRACING"] = "false"

In [71]:
pip install --upgrade "pydantic>=2.0"


Note: you may need to restart the kernel to use updated packages.


In [72]:
pip install  euriai

Note: you may need to restart the kernel to use updated packages.


In [73]:
pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [74]:
pip install -U langchain langchain-core langchain-community 

Note: you may need to restart the kernel to use updated packages.


In [75]:
import ast,re,math,os,sys
os.environ["euri-api-key"] ="euri-54b13eac3fbedb9873948ebd6e5f311f670f336cdcedb2dedf3ca864bb38d83a"
from euriai.langchain import create_chat_model
from langchain.agents import create_react_agent,AgentExecutor
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory ,InMemoryChatMessageHistory as ChatMessageHistory
from langchain_core.tools import BaseTool,tool
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    CombinedMemory,
    ReadOnlySharedMemory,
    ConversationEntityMemory

)

In [76]:
os.environ["euri-api-key"] ="euri-a5b674ba7e34a325397e4e77cc942c1784ff33ba8cbc46ae04da29087309d9aa"

In [77]:
llm_default = create_chat_model(
    api_key=os.getenv("euri-api-key"),
    model="gpt-4.1-nano",
    temperature=0.2
)


In [78]:
@tool("calculator",return_direct=True)
def calculator(expression: str) -> str:
    """Evaluate a numeric math expression . supprts + , -,*,/,**,parentheses and all kind of mathamatical functions"""
    allowed_nodes =(
        ast.Expression,
        ast.UnaryOp,
        ast.unaryop,
        ast.BinOp,
        ast.operator,
        ast.Num,
        ast.Load,
        ast.pow,
        ast.FunctionDef,
        ast.Module,
        ast.Expr,
        ast.Call,
        ast.Name,
        ast.arguments,
        ast.args,
        ast.Constant
    )
    
    allowed_names = {k:v for k,v in vars(math).items() if not k.startswith("_")}
    allowed_names.update({"abs": abs, "round": round,"min": min,"max": max})
    node = ast.parse(expression, mode="eval")
    
    
    for n in ast.walk(node):
        if not isinstance(n, allowed_nodes):
            raise ValueError(f"Expression contains invalid node {type(n)}")
        if isinstance(n, ast.Name) and n.id not in allowed_names:
            raise ValueError(f"Expression contains invalid name {n.id}")
    code = compile(node, "<string>", "eval")
    return str(eval(code, {"__builtins__": {}}, allowed_names))

In [79]:
tools_math=[calculator]

In [80]:
react_prompt_math = ChatPromptTemplate.from_messages([(
    "system","you are a precise math assistant , you can use thse tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final give me a numeric answer to the question"),
    ("human","questions: {input}\n{agent_scratchpad}")
])


In [81]:
agent_math=create_react_agent(
    llm=llm_default,
    prompt=react_prompt_math,
    tools=tools_math
)


In [82]:
memory_math = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
)


In [83]:
math_exector = AgentExecutor(
    agent=agent_math,
    tools=tools_math,
    memory=memory_math,
    verbose=True,
    handle_parsing_errors=
        (
            "you did not follow the instruction that i have given you to use the tools"
        )
)


In [84]:
@tool("kb_search", return_direct=True)
def kb_search(query: str) -> str:
    """a mock function to search from a knowledge base"""
    knowledge_base = {
        "what is the capital of france": "The capital of France is Paris.",
        "who is the president of the united states": "The president of the United States is Joe Biden.",
        "what is the largest mammal": "The largest mammal is the blue whale.",
    }
    query = query.lower().strip("?")
    for key,answer in knowledge_base.items():
        if key in query:
            return answer
    return "I don't know the answer to that question."

In [85]:
tools_kb = [kb_search]


In [86]:
react_prompt_kb = ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant that can answer question based on your knowledge base and you can use the following tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final answer to the question that you are able to search from my knowledge base"),
    ("human","questions: {input}\n{agent_scratchpad}")
])


In [87]:
mem_kb = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
    k=5
)


In [88]:
agent_kb = create_react_agent(
    llm=llm_default,
    prompt=react_prompt_kb,
    tools=tools_kb
)

In [89]:
kb_executor = AgentExecutor(
    agent=agent_kb,
    tools=tools_kb,
    memory=mem_kb,
    verbose=True,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the kb  tools"
    ),
)


In [90]:
router_prompt = ChatPromptTemplate.from_messages([
    ("system","you are a router .read the user message and out exactly one token for the input given by users:\n"
     "- MATH if user is asking for calculations \n"
     "- KB if user is asking for general knowledge questions \n"
     "output only KB or MATH nothing else from your side "),
    ("human","{input}")
])


In [91]:
router_chain = router_prompt|llm_default|StrOutputParser()

In [92]:
def _dispatcher(inputs:dict):
    """it will take new inputs from human and it will also attach hitory from memory """
    user_msg = inputs["input"]
    choice = router_chain.invoke({"input": user_msg}).strip().upper()
    print(f"router choice : {choice}")
    if choice == "MATH" or choice == "Math" or choice == "math":
        return math_exector.invoke({"input": user_msg})
    elif choice == "KB" or choice == "Kb" or choice == "kb":
        return kb_executor.invoke({"input": user_msg})
    else:
        return "I can only answer math and kb related questions"

In [93]:
dispatcher = RunnableLambda(_dispatcher)

In [94]:
_sessions = {}

In [95]:
def _get_history(session_id:str):
    if session_id not in _sessions:
        _sessions[session_id] = ChatMessageHistory()
    return _sessions[session_id]


In [96]:
orchestrator = RunnableWithMessageHistory(
    runnable=dispatcher,
    get_session_history=_get_history,
    input_key="input",
    history_messages_key="history"
)


In [97]:
cfg = {"configurable":{"session_id":"user1"}}

In [99]:
print(orchestrator.invoke({"input":"what is the capital of france?"},config=cfg))

Error in RootListenersTracer.on_llm_error callback: TracerException('No indexed run ID 26bff5f1-0a58-41dd-bbf2-d3e0d6ba94eb.')


HTTPError: 403 Client Error: Forbidden for url: https://api.euron.one/api/v1/euri/chat/completions

In [ ]:
print(orchestrator.invoke({"input":"what is cos(90)"},config=cfg))

Error in RootListenersTracer.on_llm_error callback: TracerException('No indexed run ID 4f478726-b5fa-4e9e-a37a-9232d8c5967d.')


HTTPError: 403 Client Error: Forbidden for url: https://api.euron.one/api/v1/euri/chat/completions

In [ ]:
print(orchestrator.invoke({"input":"what is (5+1)*10 "},config=cfg))

router choice : MATH


> Entering new AgentExecutor chain...
Could not parse LLM output: `thought ...  
Action by using one of the tools access that you have [calculator]  
calculator(expression: "(5+1)*10")`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the toolsCould not parse LLM output: `Questions: what is (5+1)*10  
thought: I will evaluate the expression (5+1)*10 using the calculator tool.  
Action by using one of the tools access that you have [calculator]  
calculator(expression: "(5+1)*10")`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the toolsCould not parse LLM output: `Questions: what is (5+1)*10  
thought: I will evaluate the expression (5+1)*10 using the calculator tool.  
Action by using one of the tools access that you h

In [ ]:
1.whatsapp tool
2.email tool
3.sms tool
4.telegram tool
5.calander tool
